In [3]:
from MLtools import inference
from pathlib import Path

### There are two ways to make predictions:
A. You can use the `predictions_stitching_filtering` function in the MLtools python package. <br>
B. or you can use directly the few functions provided on the Detectron2 platform (note that this step does not include any post-processing steps, and will give you only raw predictions). 

In both cases, the input image(s) need(s) to be selected and the paths to the model setup and weigths need to be specified. 

## A. Using the function *predictions_stichting_filtering*
### 1. Selection of a specific image (or multiple images)

#### Single image selection

In [ ]:
#tif = Path("/home/nilscp/tmp/tmp_pred/M1098366481.tif") 

#### Multiple images selection

In [ ]:
#input_folder = Path("/home/nilscp/tmp/tmp_pred")
#tifs = list(input_folder.rglob("*.tif"))

### 2. Setup of the model (load model setup and model weights)
Be sure that you have all of the inputs needed: 
1. Model setup file
2. Model weights
Both of those files can be downloaded from the project's GoogleDrive (please follow the instructions at: [DOWNLOAD_DATA_BOULDERING.ipynb](./DOWNLOAD_DATA_BOULDERING.ipynb))

In [4]:
home_p = Path.home()
work_dir = home_p / "tmp" / "BOULDERING"
model_dir = work_dir / "best_model"

BE CAREFUL, you have to modify the first line in the `model_setup.yaml` so that the path stored in the variable `_BASE_` corresponds to the actual path on your own computer.
```bash
_BASE_: <$HOME_DIRECTORY>/tmp/BOULDERING/best_model/base_setup.yaml
```

In [ ]:
config_file = model_dir / "model_setup.yaml"
model_weights = model_dir / "model_weights.pth" # correspond to iteration 54,000
device = "cuda" # "cpu" or "cuda", see comment below, please run the predictions for large images on a computer with a graphical card. 

I would suggest you to run this function on a computer with a graphical card (e.g., "cuda"), as the function is not yet optimized, and takes a rather long amount of time to run. I will improve this function over time, but it may take a little bit of time...

### 3. predictions_stitching_filtering

There are many steps in this function. Here is a small summary of what happens.

#### A. Predictions for image patches having slightly different strides/shifts

In order to remove predictions which are cutted at the edge of image patches (see Fig 1.), we increase the amount of overlap between image patches. There are probably different ways of doing it (which are more effective), but we are here using 6 different strides/shifts setups to cover the whole image: 
1. Stride (0, 0)  
2. Stride (block_width/2, block_height/2)
3. Stride (block_width/2, block_height)
4. Stride (block_width, block_height/2)
5. Stride (0, block_height/2)
6. Stride (block_width/2, 0)

So, if you start with an original raster/image with 200 image patches along the x-axis, and 1000 along the y-axis. There will not be only 200x1000 image patches processed but, 4x200x1000 (for the four first stride setups), and 2x(1000 + 200) for the two last setups (as the two last setups were only needed for the image patches at the edge of the original raster/image). So from a starting number of 200,000 image patches, you end up having to process 802,400 image tiles! This increase slows significantly the time it takes to process a complete single Narrow Angle Camera (NAC) or (HiRISE). We will try to improve this step in the future. <br>

Note that this steps also include the tiling of the original raster/image into smaller image patches.

<center><img src="../images/image-20230807113604998.png"/></center>

*Figure 1. Example of edge artifacts.*

#### B. Select predictions only at centres of image patches (to avoid edge predictions)

Now that we have predicted boulders for a large number of image patches, we need to only select the predictions we are interested in. The way we have done it is to select only boulders within X% from the center of patches. We used a value X = distance_p = 62.5%, allowing for a little bit of overlapping. 

**Setup 1-4** Selecting (distance_p * 100) % from centre <br>
**Setup 5-6** Selecting all of the boulders at the top and bottommost tiles which are not covered by setup 1 to 4. <br>

#### C. Remove duplicate boulders with Non-Maximum Suppresion

The last step involves removing duplicate boulders with the help of non-maximum suppression (there are lot of resource on the internet about what non-maximum suppression is, e.g., [here](https://medium.com/mlearning-ai/a-deep-dive-into-non-maximum-suppression-nms-understanding-the-math-behind-object-detection-765ff48392e5)). Long story short, it is a good way to remove duplicate/overlapping predictions. 
   
#### D. Including Test Time Augmentation (TTA) or not? 
if the flag `is_tta` is set to *True*, every image patches will be rotated in 8 potential configurations (see Fig. 2), and predictions will be made for each of the rotation. Including TTA allows for the detection of additional boulders in image patches, however, it further increases the number of processed image patches by 8 (8 x 802,400 = 6,419,200). From the test we have run, TTA is increasing the detection rate by a few %. More test need to be conducted to contrain a bit more this value. Note that Non-Maximum Suppresion is again used to remove duplicated, and keep only the predictions with the highest score/confidence. <br>

After running the function, you end up with a shapfile with no edge artificats (Fig. 3). 

<center><img src="../images/image-20230807120317504.png"/></center>

*Figure 2. 8 potential rotation transformations for an image (source: Albumentation doc).*

<center><img src="../images/predictions_150ms.gif" width="750"/></center>


*Figure 3. Example of a prediction at Courtright Reservoir (California, USA).*

#### Variables in function:
**in_raster**: path to original raster (which will eventually be tiled) <br>
**config_file**: path to config file of the Mask R-CNN model. <br>
**model_weights**: path to weights of the trained model. <br>
**device**: "cpu" or "cuda", depending if you have a graphical card on your workstation. <br>
**search_tif_pattern**: Not sure if this is useful, but it allows you to select tif files with a specific pattern, in case you have a folder with different tif names (e.g., "*.tif"). <br>
**distance_p**: Distance from centres of image patches for which predictions are selected. <br>
**block_width**: Width (in pixels) of image patches. <br>
**block_height**: Height (in pixels) of image patches.<br>
**output_dir**: Output directory for prediction shapefiles. <br>
**is_tta**: if True, 8 rotations are applied to each patch to increase the detection rate of boulders. <br>
**scores_thresh_test**: The score threshold. Predictions having scores below this threshold are automatically removed. <br>
**nms_thresh_test**: Non-Maximum Suppresion threshold. Overlapping predictions having an Intersection of Union value (IoU) above this threshold are  <br>
**min_size_test**: If you want to rescale the image (to larger dimensions). <br>
**max_size_test**: If you want to rescale the image (to larger dimensions). <br>
**pre_nms_topk_test**: Number of candidates predictions kepts before NMS is applied. <br>
**post_nms_topk_test**: Number of candidates predictions kepts after NMS is applied. <br>
**detections_per_image**: Maximum number of detections per image patch. <br>

#### Good default values
(distance_p=0.625, scores_thresh_test=0.10, nms_thresh_test=0.30, min_size_test=512, max_size_test=512, pre_nms_topk_test=2000, post_nms_topk_test=1000, detections_per_image=2000) <br>

`scores_thresh_test` and `nms_thresh_test` are the two most important values. 

#### A few tips and tricks
As the environments of different planetary bodies can be very different, I would advice you to run predictions with a low value for `scores_thresh_test`, like 0.1. And to then have a look at predictions in QGIS or ArcGIS. Try to play with the symbology of predictions so that different colors can be shown based on their scores. This will help you get a feeling of what a good cutoff threshold is for `scores_thresh_test`. From the different tests we have run, `scores_thresh_test` is usually taken between 0.2 and 0.5. Keeping `scores_thresh_test` variable allows for better boulder detection rate. <br>   

#### Going through an example
Let's download a small image (portion of NAC image XX, 9 image patches of 512x512 pixels) and run predictions on it.  

In [ ]:
url_raw_inputs = "https://drive.google.com/uc?id=10EJPATaMdS82jKOFR7rZ6o5fT6mSIhdu"
gdown.download(url_raw_inputs, (work_dir / "raw_data_BOULDERING.zip").as_posix(), quiet=True)

# only work for Linux or UNIX machine (for Windows user, you can unzip the folder manually)
!unzip ~/tmp/BOULDERING/raw_data_BOULDERING.zip -d ~/tmp/BOULDERING/

In [ ]:
home_p = Path.home()
work_dir = home_p / "tmp" / "BOULDERING"
model_dir = work_dir / "best_model"
config_file = model_dir / "model_setup_v050.yaml"
model_weights = model_dir / "model_weights.pth" # correspond to iteration 54,000
device = "cpu" # "cpu" or "cuda", see comment below, please run the predictions for large images on a computer with a graphical card. 

block_width = 512
block_height = 512
output_dir = Path("") 
is_tta = False # so that it can be run on a CPU! 
scores_thresh_test = 0.10
nms_thresh_test = 0.30
min_size_test = 512
max_size_test = 512
pre_nms_topk_test = 2000
post_nms_topk_test = 1000
detections_per_image = 2000

In [ ]:
inference.predictions_stitching_filtering(in_raster, config_file, model_weights,
                                    device, search_tif_pattern, distance_p,
                                    block_width, block_height, output_dir, is_tta=True,
                                    scores_thresh_test=0.10,
                                    nms_thresh_test=0.30,
                                    min_size_test=512, max_size_test=512,
                                    pre_nms_topk_test=2000,
                                    post_nms_topk_test=1000,
                                    detections_per_image=2000)

## B. Raw predictions with Detectron2 functions
